## 全卷积网络

一般CNN网络经过若干卷积层之后，都接全连接层，全连接层中神经元数量固定，所以这种网络需要输入固定大小的图片。

如果将全连接也替换为卷积层，就需要用到全局平均池化层，这样网络可以输入任意尺寸的图像：

> 因为经过若干卷积层之后，输出特征图像尺寸为C×H×W，对每个通道都做全局池化，一个通道就输出一个值，结果尺寸为C×1×1。则任意尺寸的特征图经过全局池化后，输出的大小都是C×1×1。

In [1]:
import time
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

### parameters

In [2]:
random_seed = 123
learning_rate = 0.001
epochs = 10
batch_size = 128

num_classes = 10

In [3]:
train_dataset = datasets.MNIST("D:/work/data/Python/mnist/",
                               train=True,
                               transform=transforms.Compose({
                                   transforms.ToTensor()
                               }),
                               download=True)
test_dataset= datasets.MNIST("D:/work/data/Python/mnist/",
                             train=False,
                             transform=transforms.Compose([
                                 transforms.ToTensor()
                             ]),
                             download=False)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# checking the dataset
for images, labels in train_loader:
    print("images shape: ", images.size())
    print("labels shape: ", labels.size())
    break

images shape:  torch.Size([128, 1, 28, 28])
labels shape:  torch.Size([128])


### model

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

class ConvNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        self.num_classes = num_classes
        
        # 28*28*1 -> 28*28*4
        self.conv1 = torch.nn.Conv2d(1, 4,
                                     kernel_size=(3, 3),
                                     stride=(1, 1),
                                     padding=1)
        # 28*28*4 -> 14*14*4
        self.conv2 = torch.nn.Conv2d(4, 4,
                                     kernel_size=(3, 3),
                                     stride=(2, 2),
                                     padding=1)
        # 14*14*4 -> 14*14*8
        self.conv3 = torch.nn.Conv2d(4, 8,
                                     kernel_size=(3, 3),
                                     stride=(1, 1),
                                     padding=1)
        # 14*14*8 -> 7*7*8
        self.conv4 = torch.nn.Conv2d(8, 8,
                                     kernel_size=(3, 3),
                                     stride=(2, 2),
                                     padding=1)
        # 7*7*8 -> 7*7*16
        self.conv5 = torch.nn.Conv2d(8, 16,
                                     kernel_size=(3, 3),
                                     stride=(1, 1),
                                     padding=1)
        # 7*7*16 -> 4*4*16
        self.conv6 = torch.nn.Conv2d(16, 16,
                                     kernel_size=(3, 3),
                                     stride=(2, 2),
                                     padding=1)
        # 4*4*16 -> 4*4*num_classes
        self.conv7 = torch.nn.Conv2d(16, self.num_classes,
                                     kernel_size=(3, 3),
                                     stride=(1, 1),
                                     padding=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.conv5(x)
        x = F.relu(x)
        x = self.conv6(x)
        x = F.relu(x)
        x = self.conv7(x)
        x = F.relu(x)
        
        # x: torch.Size([128, 10, 4, 4])
        # logits: torch.Size([128, 10, 1, 1])
        logits = F.adaptive_avg_pool2d(x, 1)
        logits.squeeze_(-1) # torch.Size([128, 10, 1])
        logits.squeeze_(-1) # torch.Size([128, 10])
        
        probas = torch.softmax(logits, dim=1)
        return logits, probas
    

torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### train

In [5]:
def accuracy(model, dataloader):
    correct, num_samples = 0, 0
    for features, targets in dataloader:
        features = features.to(device)
        targets = targets.to(device)
        
        logits, probas = model(features)
        
        _, predict_labels = torch.max(probas, 1)
        
        num_samples += targets.size(0)
        correct += (predict_labels == targets).sum()
    
    return correct.float() / num_samples

start_time = time.time()
for epoch in range(epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        features = features.to(device)
        targets = targets.to(device)
        
        # forward
        optimizer.zero_grad()
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        
        # backward
        cost.backward()
        
        # update
        optimizer.step()
        
        if (batch_idx + 1) % 50 == 0:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, epochs, batch_idx, 
                     len(train_loader), cost))

model = model.eval()
print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, epochs, 
          accuracy(model, train_loader)))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 049/469 | Cost: 2.3045
Epoch: 001/010 | Batch 099/469 | Cost: 2.2825
Epoch: 001/010 | Batch 149/469 | Cost: 2.0650
Epoch: 001/010 | Batch 199/469 | Cost: 1.8368
Epoch: 001/010 | Batch 249/469 | Cost: 1.4768
Epoch: 001/010 | Batch 299/469 | Cost: 1.2875
Epoch: 001/010 | Batch 349/469 | Cost: 1.2380
Epoch: 001/010 | Batch 399/469 | Cost: 1.2972
Epoch: 001/010 | Batch 449/469 | Cost: 1.1320
Epoch: 002/010 | Batch 049/469 | Cost: 1.3037
Epoch: 002/010 | Batch 099/469 | Cost: 1.0728
Epoch: 002/010 | Batch 149/469 | Cost: 0.8868
Epoch: 002/010 | Batch 199/469 | Cost: 1.0696
Epoch: 002/010 | Batch 249/469 | Cost: 0.8682
Epoch: 002/010 | Batch 299/469 | Cost: 1.1025
Epoch: 002/010 | Batch 349/469 | Cost: 1.1770
Epoch: 002/010 | Batch 399/469 | Cost: 0.8215
Epoch: 002/010 | Batch 449/469 | Cost: 0.7756
Epoch: 003/010 | Batch 049/469 | Cost: 0.8252
Epoch: 003/010 | Batch 099/469 | Cost: 1.1619
Epoch: 003/010 | Batch 149/469 | Cost: 0.8321
Epoch: 003/010 | Batch 199/469 | C

### evaluate

In [6]:
with torch.no_grad():
    print("Test accuracy: %.2f%%" % (accuracy(model, test_loader)))

Test accuracy: 0.76%
